# Notebook for Wine Analysis

UMN Bootcamp Project 1 - Group 6

In [1]:
import pandas as pd

In [2]:
wine_data_file = "Resources/wine_data.csv"
wine_data = pd.read_csv(wine_data_file, encoding="utf-8")

In [3]:
wine_data

,Unnamed: 0,country,description,designation,points,price (USD),province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,345,Australia,This wine contains some material over 100 year...,Rare,100,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards
1,7335,Italy,Thick as molasses and dark as caramelized brow...,Occhio di Pernice,100,210.0,Tuscany,Vin Santo di Montepulciano,NaN,NaN,NaN,Avignonesi 1995 Occhio di Pernice (Vin Santo ...,Prugnolo Gentile,Avignonesi
2,36528,France,This is a fabulous wine from the greatest Cham...,Brut,100,259.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Krug 2002 Brut (Champagne),Champagne Blend,Krug
3,39286,Italy,"A perfect wine from a classic vintage, the 200...",Masseto,100,460.0,Tuscany,Toscana,NaN,NaN,NaN,Tenuta dell'Ornellaia 2007 Masseto Merlot (Tos...,Merlot,Tenuta dell'Ornellaia
4,42197,Portugal,This is the latest release of what has long be...,Barca-Velha,100,450.0,Douro,NaN,NaN,Roger Voss,@vossroger,Casa Ferreirinha 2008 Barca-Velha Red (Douro),Portuguese Red,Casa Ferreirinha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,128258,US,This Chardonnay-Viognier blend is slightly swe...,Devil's Daughter,80,25.0,California,Contra Costa County,Central Coast,NaN,NaN,Bloomfield 2008 Devil's Daughter White (Contra...,White Blend,Bloomfield
129967,128259,US,"While there's some nice, honest fruit here, it...",NaN,80,14.0,Missouri,Missouri,NaN,NaN,NaN,Montelle 2008 Chambourcin (Missouri),Chambourcin,Montelle
129968,128260,US,"Banana, clove and orange peel open for this sw...",Lucky Twenty,80,12.0,Colorado,Colorado,NaN,NaN,NaN,BookCliff NV Lucky Twenty White (Colorado),White Blend,BookCliff
129969,128261,US,"Short and slightly sweet, there's nutty overto...",Proprietor's Reserve,80,13.0,Pennsylvania,Pennsylvania,NaN,NaN,NaN,Chaddsford 2007 Proprietor's Reserve White (Pe...,White Blend,Chaddsford


In [ ]:
# Drop region 2, taster name, taster twitter handle, and unnamed column and ensure dataframe is clean/usable


In [ ]:
# Drop any rows that do not have a price, country, rating, or year

In [ ]:
# Try to extract year from the title field

In [ ]:
# Analyze and plot year of wine vs. rating
# Scatter plot 

In [ ]:
# Analyze price of wine vs. rating
# Scatter plot

In [ ]:
# Analyze year of wine vs. price
# Scatter plot

In [ ]:
# Identify countries with the most expensive wine
# Avg wine price by country
# Include a gmap

In [ ]:
# Identify countries with the highest rated wine
# Avg wine ratings by country
# Include a gmap